# LZW Encoding And Decoding

In [1]:
def lzw_encode(data):
    dictionary = {chr(i): i for i in range(256)}
    encoded_data = []
    current_code = 256
    buffer = ''
    for symbol in data:
        if buffer + symbol in dictionary:
            buffer += symbol
        else:
            encoded_data.append(dictionary[buffer])
            dictionary[buffer + symbol] = current_code
            current_code += 1
            buffer = symbol
    if buffer:
        encoded_data.append(dictionary[buffer])
    return encoded_data

In [2]:
def lzw_decode(encoded_data):
    dictionary = {i: chr(i) for i in range(256)}
    decoded_data = []
    current_code = 256
    previous_code = encoded_data[0]
    decoded_data.append(dictionary[previous_code])
    for code in encoded_data[1:]:
        if code in dictionary:
            entry = dictionary[code]
        elif code == current_code:
            entry = dictionary[previous_code] + dictionary[previous_code][0]
        else:
            raise ValueError("Bad compressed code")

        decoded_data.append(entry)
        dictionary[current_code] = dictionary[previous_code] + entry[0]
        current_code += 1
        previous_code = code
    return ''.join(decoded_data)

In [3]:
original_text = "TOBEORNOTTOBEORTOBEORNOT"
encoded_text = lzw_encode(original_text)
print("Encoded data:", encoded_text)

Encoded data: [84, 79, 66, 69, 79, 82, 78, 79, 84, 256, 258, 260, 265, 259, 261, 263]


In [4]:
decoded_text = lzw_decode(encoded_text)
print("Decoded data:", decoded_text)

Decoded data: TOBEORNOTTOBEORTOBEORNOT


# Arithmetic Encoding 

In [5]:
class ArithmeticEncoder:
    def __init__(self):
        self.low = 0
        self.high = 1
        self.range = 1

    def encode(self, data, probabilities):
        for symbol in data:
            symbol_range = probabilities[symbol]
            self.update_range(symbol_range)
        return self.low

    def update_range(self, symbol_range):
        range_size = self.high - self.low
        self.high = self.low + range_size * symbol_range[1]
        self.low = self.low + range_size * symbol_range[0]

    def normalize_range(self):
        while True:
            if self.high < 0.5:
                self.low *= 2
                self.high *= 2
                self.range *= 2
            elif self.low >= 0.5:
                self.low = 2 * (self.low - 0.5)
                self.high = 2 * (self.high - 0.5)
                self.range *= 2
            elif 0.25 <= self.low < 0.75 and 0.25 < self.high <= 0.75:
                self.low = 2 * (self.low - 0.25)
                self.high = 2 * (self.high - 0.25)
                self.range *= 2
            else:
                break

    def get_encoded_value(self, data, probabilities):
        self.low = 0
        self.high = 1
        self.range = 1
        encoded_value = self.encode(data, probabilities)
        self.normalize_range()
        return encoded_value


In [6]:
class ArithmeticDecoder:
    def __init__(self):
        self.code = 0
        self.low = 0
        self.high = 1

    def decode(self, encoded_value, data_length, probabilities):
        decoded_data = []
        self.code = encoded_value
        for _ in range(data_length):
            symbol = self.decode_symbol(probabilities)
            decoded_data.append(symbol)
        return decoded_data

    def decode_symbol(self, probabilities):
        range_size = self.high - self.low
        for symbol, (low_range, high_range) in probabilities.items():
            symbol_range_size = high_range - low_range
            if (self.code - self.low) / range_size <= high_range and (self.code - self.low) / range_size > low_range:
                self.low = self.low + range_size * low_range
                self.high = self.low + range_size * symbol_range_size
                return symbol

    def normalize_range(self):
        while True:
            if self.high < 0.5:
                self.low *= 2
                self.high *= 2
            elif self.low >= 0.5:
                self.low = 2 * (self.low - 0.5)
                self.high = 2 * (self.high - 0.5)
            elif 0.25 <= self.low < 0.75 and 0.25 < self.high <= 0.75:
                self.low = 2 * (self.low - 0.25)
                self.high = 2 * (self.high - 0.25)
            else:
                break

    def get_decoded_data(self, encoded_value, data_length, probabilities):
        self.low = 0
        self.high = 1
        self.code = encoded_value
        decoded_data = self.decode(encoded_value, data_length, probabilities)
        self.normalize_range()
        return decoded_data

In [7]:
data = "ABBABBABBA"
probabilities = {'A': (0, 0.5), 'B': (0.5, 1)}

In [8]:
encoder = ArithmeticEncoder()
encoded_value = encoder.get_encoded_value(data, probabilities)
print("Encoded value:", encoded_value)

Encoded value: 0.427734375


In [9]:
decoder = ArithmeticDecoder()
decoded_data = decoder.get_decoded_data(encoded_value, len(data), probabilities)
print("Decoded data:", ''.join(decoded_data))

Decoded data: ABBABBABAB
